In [1]:
import pandas as pd
import numpy as np

In [2]:
production_tb = pd.read_csv('./data/production.csv', encoding='UTF-8')

In [3]:
monthly_index_tb = pd.read_csv('./data/monthly_index.csv', encoding='UTF-8')

## 5. 분할
### 5.1. 모델 검증을 위한 레코드 분할
#### 교차 검증
홀드아웃 검증

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

train_data, test_data, train_target, test_target = \
    train_test_split(production_tb.drop('fault_flg', axis=1),
                     production_tb[['fault_flg']],
                     test_size=0.2)

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)
test_target.reset_index(inplace=True, drop=True)

In [5]:
row_no_list = list(range(len(train_target)))
k_fold = KFold(n_splits=4, shuffle=True)

for train_cv_no, test_cv_no in k_fold.split(row_no_list):
    train_cv = train_data.iloc[train_cv_no, :]
    test_cv = train_data.iloc[test_cv_no, :]

### 5.2. 모델 검증을 위한 시간 데이터 분할
학습 기간이 일정한 패턴

 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12
--|--|--|--|--|--|--|--|--|--|--|--
학습|학습|학습|학습|학습|학습|검증|검증|  |  |  |  
  |  |학습|학습|학습|학습|학습|학습|검증|검증|  |  
  |  |  |  |학습|학습|학습|학습|학습|학습|검증|검증

학습 기간이 늘어나는 패턴

 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12
--|--|--|--|--|--|--|--|--|--|--|--
학습|학습|학습|학습|학습|학습|검증|검증|  |  |  |  
학습|학습|학습|학습|학습|학습|학습|학습|검증|검증|  |  
학습|학습|학습|학습|학습|학습|학습|학습|학습|학습|검증|검증


#### 학습 및 검증을 위한 시간 데이터 준비

In [6]:
train_window_start = 1
train_window_end = 24
horizon = 12
skip = 12

monthly_index_tb.sort_values(by='year_month')

while True:
    test_window_end = train_window_end + horizon
    
    print(train_window_start, train_window_end, train_window_end+1, test_window_end)
    
    train = monthly_index_tb[train_window_start:train_window_end]
    test = monthly_index_tb[(train_window_end+1):test_window_end]
    
    if test_window_end >= len(monthly_index_tb.index):
        break
        
    train_window_start += skip
    train_window_end += skip

1 24 25 36
13 36 37 48
25 48 49 60
37 60 61 72
49 72 73 84
61 84 85 96
73 96 97 108
85 108 109 120
